In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

import time
import pandas as pd

In [ ]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('window-size= 1920,1080')
chrome_options.add_argument('--kiosk')

# executable_path : /opt/ml/crawling/chromedriver
driver = webdriver.Chrome(executable_path='/opt/ml/input/crawings/chromedriver', chrome_options=chrome_options)

In [ ]:
data = pd.read_csv('./save_csv/river_behind_concat.csv')
url_list = list(data['url'].values)
url_list[:5]

In [ ]:
len(url_list)
userlink = pd.DataFrame()

In [ ]:
# from tqdm import tqdm
# #url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]

# userlink = pd.DataFrame()
# for _url in tqdm(url_list[20:30]): # [0:500]
#     URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
#     driver.get(URL)
#     time.sleep(2.5)
#     prev_height = driver.execute_script("return document. body.scrollHeight")
#     while True:
#         driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
#         time.sleep(0.5)
#         try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
#         except: pass
#         current_height = driver.execute_script("return document. body.scrollHeight")
#         if current_height == prev_height: break
#         prev_height = current_height
#         time.sleep(0.1)
    
#     link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
#     link_list = [link.get_attribute("href") for link in link_list]
#     user_list = driver.find_elements(By.CLASS_NAME, "sBWyy")
#     user_list = [user.text for user in user_list]
#     review_time_list = driver.find_elements(By.CLASS_NAME, "sb8UA")
#     review_time_list = [reviews.text for reviews in review_time_list]
#     length = min(len(user_list), len(link_list), len(review_time_list))
#     userlink2 = pd.DataFrame({'user' : user_list[:length], 'link' : link_list[:length], 'review_time' : review_time_list[:length]}, dtype = str)
#     userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)

In [ ]:
from tqdm import tqdm
#url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]
current_status = 0

for _url in tqdm(url_list[0:600]):
    if current_status % 10 == 0: #크롬 드라이버 가끔 이상해지는거때문에 50번마다 재실행
        #driver = webdriver.Chrome(executable_path='/opt/ml/input/crawings/chromedriver', chrome_options=chrome_options)
        #action = ActionChains(driver)
        userlink.to_csv('./user_csv/river_behind500.csv', index=False)#river_behind500
    
    driver = webdriver.Chrome(executable_path='/opt/ml/input/crawings/chromedriver', chrome_options=chrome_options)
    action = ActionChains(driver)
    #URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    #_url = 32548015 
    print(_url)
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    #prev_height = driver.execute_script("return document. body.scrollHeight")
    count =0
    while True:        
        try: action.move_to_element(driver.find_element(By.CLASS_NAME, "lfH3O")).click().perform() 
        except: break
        print("\r",count, end="")
        count+= 1
    
    print("click 1/2 complete")
    time.sleep(2.5)

    try: 
        #action.move_to_element(driver.find_elements(By.CLASS_NAME, "YeINN")[-1]).perform()  #선택 리뷰 창 + 버튼 누르기
        driver.find_element(By.CLASS_NAME, 'I8cuq').click()
    except: print("NO 선택리뷰창")
    time.sleep(2.5)
    action = ActionChains(driver)
    while True:
        try: 
            action.move_to_element(driver.find_element(By.CLASS_NAME, "lfH3O")).click().perform() 
            #driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: 
            break
        print("\r",count, end="")
        count+= 1
    print("click 2/2 complete")
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    user = soup.find_all(class_='YeINN')

    link_list = [i.a['href'] for i in user]
    user_list = [i.text for i in user]

    #time.sleep(5)
   
    print(_url, len(link_list), len(user_list))
    userlink2 = pd.DataFrame({'link' : link_list, 'user' : user_list}, dtype = str)
    userlink2['rest'] = _url
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)
    current_status += 1
    break
    

In [ ]:
userlink

In [ ]:
userlink.to_csv('./user_csv/river_behind500.csv', index=False)